In [1]:
import pandas as pd
import os, random, sys, time
from urllib.parse import urlparse
from selenium import webdriver
from bs4 import BeautifulSoup
import re

1. Abrimos una pestaña en el browser que será comandada de manera automática

In [2]:
browser = webdriver.Chrome('driver/chromedriver.exe')

2. Abrimos la página de LinkedIn y nos logeamos

In [3]:
browser.get('https://www.linkedin.com/uas/login') #carga de página
file = open('config.txt') #se debe configurar un path a un txt con el usuario y la contraseña
lines = file.readlines() #leemos el archivo
username = lines[0] #Obtenemos el usuario
password = lines[1] #Obtenemos la contraseña
elementID = browser.find_element_by_id('username') #Buscamos el campo donde introducir el username
elementID.send_keys(username) #Pasamos el usuario
elementID = browser.find_element_by_id('password') #Buscamos el campo donde introducir el username
elementID.send_keys(password) #Pasamos la contraseña
elementID.submit() #Apretamos el boton de login

3. Creamos el scrapper

In [4]:
def feature_extraction(full_link,n_post): #los argumentos de la función son los perfiles a explorar y el número de post
    n_scroll = max(int(n_post/5 - 1),1) #En base al número de post calculamos cuantos scrolls debemos realizar en la página
    fullLink = full_link + 'detail/recent-activity/' #Añadimos un sufijo al link para que nos lleve directamente a la actividad del perfil
    browser.get(fullLink) #Obtenemos la información de la página

    # Scroll down en la página web
    last_height = browser.execute_script('return document.body.scrollHeight') #Le pedimos obtener la información de la página
    for i in range(n_scroll): #Recorremos el número de scrolls 
        browser.execute_script('window.scrollTo(0,document.body.scrollHeight);')
        time.sleep(random.uniform(5,7)) #Durante el scroll dormimos por unos segundos al bot
        new_height = browser.execute_script('return document.body.scrollHeight') #Guardamos la información extraída
        if new_height == last_height:
            break
        last_height = new_height

    src = browser.page_source
    soup = BeautifulSoup(src,'lxml')

    # Busco la sección del header
    followers_div = soup.find('div',{'class':'flex-1 text-align-right t-14 t-black t-light'})
    followers = followers_div.get_text().strip()
    name_div = soup.find('h3',{'class':'single-line-truncate t-16 t-black t-bold mt2'})
    name = name_div.get_text().strip()
    text_div = soup.find_all('div',{'class':'feed-shared-update-v2__description-wrapper ember-view'})
    interaction_div = soup.find_all('ul',{'class':'social-details-social-counts ember-view'})
    post_div = soup.find_all('div',{'class':'occludable-update ember-view'})
    
    # Name and followers
    name_list = [name]*len(text_div)
    follower_list = [followers]*len(text_div)

    # Otras variables
    embedding = []
    header = []
    reaction = []
    comment = []
    image_cont = []
    linkedin_video_cont = []
    external_video_cont = []
    podcast_cont = []
    article_cont = []

    for i in range(0,len(text_div)):
        
        # text
        try:
            text = text_div[i].find('span',{'dir':'ltr'})
        except:
            text = 0
            
        # embeddings
        try:
            embed = text.find_all('a', href=True)
            embedding.append(''.join(str(embed)))
        except:
            embedding.append(0)
        
        # header
        try:
            head = text.get_text().strip()
            header.append(head)
        except:
            header.append(0)
            
        # reactions
        try:
            reaction_div = interaction_div[i].find('span',{'class':'v-align-middle social-details-social-counts__reactions-count'})
            reaction.append(reaction_div.get_text().strip())
        except:
            reaction.append('Disable')
        
        # comments
        try:
            comment_div = interaction_div[i].find_all('span',{'class':'v-align-middle'})[1]
            comment.append(comment_div.get_text().strip(' Comments'))
        except:
            comment.append('Disable')
            
        # image
        try:
            image = re.search(r'(feed-shared-image)',str(post_div[i]))[0]
            image_cont.append(image)
        except:
            image_cont.append(0)
        # linkedin_video
        try:
            linkedin_video = re.search(r'(feed-shared-linkedin-video)',str(post_div[i]))[0]
            linkedin_video_cont.append(linkedin_video)
        except:
            linkedin_video_cont.append(0)
        # external_video
        try:
            external_video = re.search(r'(feed-shared-external-video)',str(post_div[i]))[0]
            external_video_cont.append(external_video)
        except:
            external_video_cont.append(0)
        # podcast
        try:
            podcast = re.search(r'(podcast)',str(post_div[i]))[0]
            podcast_cont.append(podcast)
        except:
            podcast_cont.append(0)
        # article
        try:
            article = re.search(r'(article)',str(post_div[i]))[0]
            article_cont.append(article)
        except:
            article_cont.append(0)

    
    # Creamos un data frame con los datos
    df = pd.DataFrame({'Name':name_list,
                       'Followers':follower_list,
                       'Embedding': embedding,
                       'Header': header,
                       'Comment': comment,
                       'Reaction':reaction,
                       'Image':image_cont,
                       'LinkedIn_Video':linkedin_video_cont,
                       'External_Video':external_video_cont,
                       'Podcast':podcast_cont,
                       'Article':article_cont})
    return(df)

4. Leemos el archivo que tiene el listado de links con los perfiles de los influencers de LinkedIn

In [ ]:
df_links = pd.read_csv('url.csv')
list_links = df_links.url.tolist()

5. Ejecutamos el scrapper y recorremos la lista de influencers extrayendo sus posteos

In [ ]:
%%time
df_final = pd.DataFrame({'Name':[],'Followers':[],'Embedding': [],'Header': [],'Comment': [],
                         'Reaction':[],'Image':[],'LinkedIn_Video':[],'External_Video':[],'Podcast':[],'Article':[]})
for i in list_links:
    df = feature_extraction(i,100)
    df_final = pd.concat([df_final,df])
    df_final.reset_index(drop=True,inplace=True)
df_final

6. Guardamos el dataframe obtenido

In [ ]:
df_final.to_csv('influencer_1_100.csv',index=False)